In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("biometria.csv")
df.replace("V", "volei", inplace=True)
df.replace("F", "futebol", inplace=True)
df.head()

,Esporte,Altura (m),Peso (kg)
0,volei,1.90,76
1,volei,1.98,70
2,volei,2.04,101
3,volei,1.88,77
4,volei,2.14,109


In [ ]:
def classificador_limiar(limiar, atributo, dados, rotulos=None):
    predicoes = []
    for idx in range(len(dados)):
        if dados.iloc[idx][atributo] <= limiar:
            predicoes.append(rotulos[0])
        else:
            predicoes.append(rotulos[1])
    return pd.DataFrame(predicoes, columns=['Esporte'])

In [ ]:
X = df[["Altura (m)", "Peso (kg)"]]
y = df[["Esporte"]]

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True, random_state=10)


In [ ]:
X_treino.shape, X_teste.shape, y_treino.shape, y_teste.shape

((36, 2), (9, 2), (36, 1), (9, 1))

In [ ]:
def treinar_classificador_limiar(n_limiares, atributo, rotulos, dados_X, dados_y):
    alt_max = np.max(dados_X[atributo])
    alt_min = np.min(dados_X[atributo])
    limiares = np.linspace(alt_min, alt_max, n_limiares)

    acuracias = []
    melhor_limiar = None
    melhor_acuracia = -1

    for limiar in limiares:
        predicoes = classificador_limiar(limiar, atributo, dados_X, rotulos)
        acuracia = accuracy_score(dados_y, predicoes['Esporte'])

        if acuracia > melhor_acuracia:
            melhor_acuracia = acuracia
            melhor_limiar = limiar

        acuracias.append(acuracia)

    return melhor_limiar, melhor_acuracia, acuracias

In [ ]:
limiar, acc, _ = treinar_classificador_limiar(100, 'Altura (m)', ["futebol", "volei"], X_treino, y_treino)

In [ ]:
limiar, acc, y_teste, X_teste

(1.8923232323232324,
 0.8888888888888888,
     Esporte
 31  futebol
 37  futebol
 8     volei
 26  futebol
 17    volei
 22  futebol
 4     volei
 14    volei
 39  futebol,
     Altura (m)  Peso (kg)
 31        1.88         78
 37        1.80         75
 8         1.95         79
 26        1.84         79
 17        2.03         90
 22        1.86         80
 4         2.14        109
 14        1.94         76
 39        1.80         67)

In [ ]:
predicoes = classificador_limiar(limiar, "Altura (m)", X_teste, ["futebol", "volei"])

accuracy_score(y_teste['Esporte'], predicoes["Esporte"])

1.0

## Estimando o desempenho do modelo usando simulaçãode Montecarlo

In [1]:
acuracias = []
n_iteracoes = 100

for i in range(n_iteracoes):
  X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True, random_state=i)

  limiar, acc, _ = treinar_classificador_limiar(100, 'Altura (m)', ["futebol", "volei"], X_treino, y_treino)

  predicoes = classificador_limiar(limiar, "Altura (m)", X_teste, ["futebol", "volei"])

  acuracias.append(accuracy_score(y_teste['Esporte'], predicoes["Esporte"]))

NameError: ignored

In [ ]:
med, desv = np.mean(acuracias), np.std(acuracias)
min, max = np.min(acuracias), np.max(acuracias)

print("Acurácia Média: %.2f +- %.2f (min: %.2f, max: %.2f)" % (med, desv, min, max))

Acurácia Média: 0.87 +- 0.12 (min: 0.56, max: 1.00)
